In [1]:
import os
import glob
import cv2
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import keras
import keras_tuner as kt
import datetime
import tensorflow as tf
from sklearn.metrics import confusion_matrix

2023-01-09 16:19:14.402419: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 16:19:14.473736: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-09 16:19:14.491415: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-09 16:19:14.833957: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
data_dir = os.path.join("..", "yolo_preperation", "data")

In [3]:
shrink_factor = 4

In [4]:
def import_yolo_data(data_dir, split, stride=1):
    image_path = os.path.join(data_dir, "images", split)
    label_path = os.path.join(data_dir, "labels", split)
    
    images = []
    image_filenames= []
    labels = []
    
    i = 0
    
    for file_name in glob.glob(os.path.join(image_path, "*.jpg")):
        if i % stride == 0:
            img = cv2.imread(file_name)
            height, width = img.shape[:2]
            img = cv2.resize(img, (int(width / shrink_factor), int(height / shrink_factor)))
            images.append(img / 255.0)
            
            image_filenames.append(file_name)
        i += 1
        
        
#     i = 0
    
#     for label_txt in glob.glob(os.path.join(label_path, "*.txt")):
#         if i % stride == 0:
#             with open(label_txt) as f:
#                 labels.append(int(f.readlines()[0].split(" ")[0]))
#         i += 1
    for image_filename in image_filenames:
        with open(image_filename.replace("images", "labels", 1).replace("jpg", "txt", 1)) as f:
            labels.append(int(f.readlines()[0].split(" ")[0]))
            
    return (images, labels)

In [5]:
stride = 4
train_X, train_y = import_yolo_data(data_dir, "train", stride)
val_X, val_y = import_yolo_data(data_dir, "val", stride)
test_X, test_y = import_yolo_data(data_dir, "test", stride)

print(len(train_X))

600


In [6]:
train_y = OneHotEncoder().fit_transform(np.array(train_y).reshape(-1,1)).reshape(-1, 4).toarray()
val_y = OneHotEncoder().fit_transform(np.array(val_y).reshape(-1,1)).reshape(-1, 4).toarray()
test_y = OneHotEncoder().fit_transform(np.array(test_y).reshape(-1,1)).reshape(-1, 4).toarray()

class_names = ["0", "1", "2", "3"]

In [7]:
print(len(train_X))
print(len(train_y))
print()
print(len(val_X))
print(len(val_y))
print()
print(len(test_X))
print(len(test_y))

600
600

200
200

200
200


In [8]:
train_X = np.array(train_X)
val_X = np.array(val_X)
test_X = np.array(test_X)

In [9]:
def model_builder(hp):

    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, (3, 3), input_shape = (int(720 / shrink_factor), int(1280 / shrink_factor), 3), activation = 'relu'))
    
    model.add(keras.layers.MaxPooling2D(2))
    model.add(keras.layers.Conv2D(32, (3, 3), activation = 'relu'))
    model.add(keras.layers.MaxPooling2D(2))
    model.add(keras.layers.Conv2D(64, (3, 3), activation = 'relu'))
    model.add(keras.layers.MaxPooling2D(2))
    model.add(keras.layers.Conv2D(64, (3, 3), activation = 'relu'))
    model.add(keras.layers.MaxPooling2D(2))
    
    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(128, activation = 'relu'))
    model.add(keras.layers.Dense(4, activation = 'softmax'))
    
    hp_learning_rate = hp.Choice('learning_rate', values = [0.5, 1e-1, 1e-2, 1e-3, 1e-4])
#     hp_learning_rate = 0.01
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="auto",
    name="categorical_crossentropy",
), optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), metrics=['accuracy'])
    return model

In [10]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=15,
                     factor=3,
                     directory='kt_cnn',
                     project_name='cnnv3_4')

2023-01-09 16:19:25.944506: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-09 16:19:26.076660: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-09 16:19:26.076703: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-09 16:19:26.077334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the ap

In [11]:
logdir = os.path.join(
    "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir)

In [12]:
tuner.search(train_X, train_y,
             validation_data=(val_X, val_y),
             callbacks=[tensorboard_callback])

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.2800000011920929

Best val_accuracy So Far: 0.35499998927116394
Total elapsed time: 00h 00m 15s
INFO:tensorflow:Oracle triggered exit


In [13]:
best_hps = tuner.get_best_hyperparameters()[0]

In [14]:
best_model = tuner.get_best_models()[0]
best_model.summary()
best_model.evaluate(val_X, val_y)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 318, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 159, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 157, 32)       9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 78, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 76, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 38, 64)       0

[1.3856960535049438, 0.35499998927116394]

In [15]:
y_pred = np.argmax(best_model.predict(val_X), axis=1)

7/7 [==============================] - 0s 8ms/step


In [17]:
best_model.predict(val_X)

7/7 [==============================] - 0s 29ms/step


array([[0.23437607, 0.26372686, 0.26218462, 0.23971242],
       [0.2313647 , 0.26107207, 0.2658053 , 0.24175788],
       [0.23358706, 0.26656336, 0.26284584, 0.23700368],
       [0.23176597, 0.26090354, 0.26981857, 0.23751192],
       [0.2375979 , 0.27240294, 0.26139525, 0.22860393],
       [0.23569083, 0.2638172 , 0.26174635, 0.23874567],
       [0.23502721, 0.2609193 , 0.26400158, 0.24005191],
       [0.23368423, 0.26086992, 0.2654308 , 0.24001507],
       [0.23338601, 0.2694043 , 0.25967634, 0.23753339],
       [0.23735717, 0.2620117 , 0.26209062, 0.23854052],
       [0.22978458, 0.25945103, 0.26993364, 0.24083078],
       [0.23205325, 0.2608638 , 0.26714584, 0.23993717],
       [0.23470336, 0.26103547, 0.26675346, 0.2375077 ],
       [0.23190506, 0.26239052, 0.26568568, 0.24001876],
       [0.23162213, 0.27025312, 0.2614597 , 0.23666508],
       [0.23347314, 0.26160973, 0.26666132, 0.23825584],
       [0.23640564, 0.26079124, 0.26216298, 0.24064016],
       [0.23468709, 0.26126742,